# Sagemaker RL Lab - Summit  - "One Click!"

Hi! Run the code cell below:



1.  First select the code cell by simply left clicking on it below, when you select it - it should go green as follows:

![alt text](https://i.ibb.co/rt9NdXL/Screen-Shot-2019-03-25-at-2-02-08-pm.png)

2.   Once the cell is selected, click the "run" button at the top of the notebook, which looks like this:

![alt text](https://i.ibb.co/fCwfvBk/Screen-Shot-2019-03-25-at-2-05-33-pm.png)

This will start our model's training process. We will spend the rest of this lab explaining what is going on here line by line - with an amazing result to follow!

Please open the additional notebook "Sagemaker RL Lab - Summit 2019 - Explanation" now!



In [0]:
!pip install ray[rllib]
!pip install ray[debug]

import ray
import ray.rllib.agents.impala as impala
from ray.tune.logger import pretty_print

ray.init()

config = impala.DEFAULT_CONFIG.copy()
config["num_gpus"] = 1
config["num_workers"] = 2
agent = impala.ImpalaAgent(config=config, env="BreakoutDeterministic-v4") 


for i in range(10000):
   # Perform one iteration of training the policy with IMPALA
   result = agent.train()
   print(pretty_print(result))

   if i % 100 == 0:
       checkpoint = agent.save()
       print("checkpoint saved at", checkpoint)

    100% |████████████████████████████████| 75.0MB 593kB/s 
    100% |████████████████████████████████| 71kB 24.6MB/s 
    100% |████████████████████████████████| 18.8MB 2.7MB/s 
    100% |████████████████████████████████| 368kB 23.7MB/s 
    100% |████████████████████████████████| 2.9MB 12.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/e6/b1/a6/9719530228e258eba904501fef99d5d85c80d52bd8f14438a3
Successfully built setproctitle


2019-03-25 03:11:34,364	INFO node.py:439 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-25_03-11-34_206/logs.
2019-03-25 03:11:34,480	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:18239 to respond...
2019-03-25 03:11:34,616	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:43755 to respond...
2019-03-25 03:11:34,624	INFO services.py:761 -- Starting Redis shard with 2.52 GB max memory.
2019-03-25 03:11:34,673	INFO services.py:1449 -- Starting the Plasma object store with 3.78 GB memory using /dev/shm.
2019-03-25 03:11:35,951	INFO policy_evaluator.py:275 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
(pid=297) 2019-03-25 03:11:46,232	INFO policy_evaluator.py:275 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
(pid=297) 2019-03-25 03:11:46.241333: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=297) 2019-03-25 03:11:46.241599: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x6037b80 executing computations on platform Host. Devices:
(pid=297) 2019-03-25 03:11:46.241631: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
(pid=297) 2019-03-25 03:11:46.246452: E tensorflow/stream_executor/cuda/cuda_driver.cc:300] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(pid=297) 2019-03-25 03:11:46.246530: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:161] retrieving CUDA diagnostic information for host: 228cabdf7ba8
(pid=297) 2019-03-

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


custom_metrics: {}
date: 2019-03-25_03-11-50
done: false
episode_len_mean: 144.5
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 4
episodes_total: 4
experiment_id: 7d8b7b53fcf743bb8fe8f167daf72824
hostname: 228cabdf7ba8
info:
  learner_queue:
    size_count: 0
    size_mean: .nan
    size_quantiles: []
    size_std: .nan
  num_steps_replayed: 0
  num_steps_sampled: 400
  num_steps_trained: 0
  num_weight_syncs: 8
  sample_throughput: 36.58
  timing_breakdown:
    learner_dequeue_time_ms: .nan
    learner_grad_time_ms: .nan
    learner_load_time_ms: .nan
    learner_load_wait_time_ms: .nan
    optimizer_step_time_ms: 13.627
iterations_since_restore: 1
node_ip: 172.28.0.2
num_metric_batches_dropped: 0
off_policy_estimator: {}
pid: 206
policy_reward_mean: {}
time_since_restore: 10.48203182220459
time_this_iter_s: 10.48203182220459
time_total_s: 10.48203182220459
timestamp: 1553483510
timesteps_since_restore: 400
timesteps_this_iter: 400
timeste

/usr/local/lib/python3.6/dist-packages/ray/tune/logger.py:258: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(value, float):


custom_metrics: {}
date: 2019-03-25_03-12-01
done: false
episode_len_mean: 158.66666666666666
episode_reward_max: 2.0
episode_reward_mean: 0.5
episode_reward_min: 0.0
episodes_this_iter: 20
episodes_total: 24
experiment_id: 7d8b7b53fcf743bb8fe8f167daf72824
hostname: 228cabdf7ba8
info:
  learner:
    cur_lr: 0.0005000000237487257
    entropy: 0.0
    grad_gnorm: 40.000003814697266
    max_KL: 0.0
    mean_KL: 0.0
    median_KL: 0.0
    model: {}
    policy_loss: -0.0
    var_gnorm: 9.722892761230469
    vf_explained_var: 0.000356137752532959
    vf_loss: 1912592.25
  learner_queue:
    size_count: 6
    size_mean: 1.0
    size_quantiles:
    - 0.0
    - 0.0
    - 0.5
    - 2.5
    - 3.0
    size_std: 1.1547005383792515
  num_steps_replayed: 0
  num_steps_sampled: 3100
  num_steps_trained: 3000
  num_weight_syncs: 62
  sample_throughput: 258.159
  timing_breakdown:
    learner_dequeue_time_ms: 2292.355
    learner_grad_time_ms: 1142.885
    learner_load_time_ms: .nan
    learner_load_wai